In [1]:
!dir "../data"

 驱动器 C 中的卷没有标签。
 卷的序列号是 D4A3-95FD

 C:\Users\Mark\1AI.NLP\AI.NLP\data 的目录

2019-02-09  21:46    <DIR>          .
2019-02-09  21:46    <DIR>          ..
2017-12-03  12:09       160,966,723 sqlResult_1558435.csv
               1 个文件    160,966,723 字节
               2 个目录 21,873,483,776 可用字节


In [2]:
def cut(string): return ' '.join(jieba.cut(string))

In [3]:
import pandas as pd
import jieba

In [4]:
data_csv = pd.read_csv('../data/sqlResult_1558435.csv', header=0, encoding='gb18030')

In [5]:
data_csv.columns

Index(['id', 'author', 'source', 'content', 'feature', 'title', 'url'], dtype='object')

In [6]:
data_csv['content'][1].replace('\r\n','')

'骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考虑性能而去屏蔽掉小核心。相反，他们正联手微软，找到一种适合桌面平台的、兼顾性能和功耗的完美方案。报道称，微软已经拿到了一些新的源码，以便Windows 10更好地理解big.little架构。资料显示，骁龙835作为一款集成了CPU、GPU、基带、蓝牙/Wi-Fi的SoC，比传统的Wintel方案可以节省至少30%的PCB空间。按计划，今年Q4，华硕、惠普、联想将首发骁龙835 Win10电脑，预计均是二合一形态的产品。当然，高通骁龙只是个开始，未来也许还能见到三星Exynos、联发科、华为麒麟、小米澎湃等进入Windows 10桌面平台。'

In [7]:
import json

obj = json.loads(data_csv['feature'][4])

In [8]:
obj['site']

'网易热门'

In [9]:
def get_top_corpus_source(n):
    corpus,sources=[],[]
    for i in range(n):
        if isinstance(data_csv['content'][i], float): # 排除float型空行
            corpus.append('')
            sources.append('x')
            continue
            
        corpus.append(cut(data_csv['content'][i].replace('\r\n','').replace(u'\u3000','')))
        if 'site' in data_csv['feature'][i]: sources.append(json.loads(data_csv['feature'][i])['site'])
        else: sources.append("x")
    return (corpus, sources)

In [10]:
c_s = get_top_corpus_source(100)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Mark\AppData\Local\Temp\jieba.cache
Loading model cost 0.837 seconds.
Prefix dict has been built succesfully.


In [11]:
corpus = c_s[0]

In [12]:
len(corpus)

100

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer()

In [15]:
tfidf_vect = vectorizer.fit_transform(c_s[0])

In [16]:
words = vectorizer.get_feature_names()

In [17]:
tfidf_ary = tfidf_vect.toarray()

In [18]:
tfidf_ary.shape

(100, 9102)

In [20]:
tfidf_ary.transpose().shape

(9102, 100)

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
logReg = LogisticRegression()

In [23]:
train_rate = 0.7

In [24]:
train_index = int(train_rate * len(c_s[0]))
train_X = tfidf_ary[:train_index]
train_Y = c_s[1][:train_index]

test_X = tfidf_ary[train_index:]
test_Y = c_s[1][train_index:]

In [48]:
logReg.fit(train_X, train_Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
logReg.predict(test_X)

array(['cnbeta', '环球', '环球', 'cnbeta', '环球', 'cnbeta', 'cnbeta', 'cnbeta',
       '环球', 'cnbeta', '环球', 'cnbeta', 'cnbeta', 'cnbeta', 'cnbeta', '环球',
       'cnbeta', 'cnbeta', 'cnbeta', 'cnbeta', 'cnbeta', 'cnbeta',
       'cnbeta', 'cnbeta', 'cnbeta', 'cnbeta', 'cnbeta', 'cnbeta',
       'cnbeta', 'cnbeta'], dtype='<U6')

In [49]:
train_X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.06353837],
       [0.        , 0.23259552, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [52]:
import json

import jieba
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


def cut(content):
    return ' '.join(jieba.cut(content))


def get_top_corpus_source(n, filename='../data/sqlResult_1558435.csv', encoding='gb18030'):
    data_csv = pd.read_csv(filename, header=0, encoding=encoding)
    corpus, sources = [], []
    for i in range(n):
        if isinstance(data_csv['content'][i], float):  # 排除float型空行
            corpus.append('')
            sources.append('x')
            continue

        corpus.append(cut(data_csv['content'][i].replace('\r\n', '').replace(u'\u3000', '')))
        if 'site' in data_csv['feature'][i]:
            sources.append(json.loads(data_csv['feature'][i])['site'])
        else:
            sources.append("x")
    return corpus, sources


def tf_idf(data_set):
    vectorizer = TfidfVectorizer()
    vector = vectorizer.fit_transform(data_set[0])
    return vector.toarray(), data_set[1]


def split_data_set(data_set, training_rate=0.7):
    train_index = int(training_rate * len(data_set[1]))
    train_x = data_set[0][:train_index]
    train_y = data_set[1][:train_index]

    test_x = data_set[0][train_index:]
    test_y = data_set[1][train_index:]
    return train_x, train_y, test_x, test_y


def train_by_lr(train_x, train_y):
    lr = LogisticRegression()
    lr.fit(train_x, train_y)
    return lr


def train(method_func, train_x, train_y):
    method_func.fit(train_x, train_y)
    return method_func


def predict(method_func, test_x):
    return method_func.predict(test_x)


corpus_source = get_top_corpus_source(100)



In [53]:
ds = tf_idf(corpus_source)

In [54]:
tt_data = split_data_set(ds)

In [57]:
lr_recognizer = train_by_lr(tt_data[0],tt_data[1])
predict_rs = predict(lr_recognizer, tt_data[2])
print(str(predict_rs))
print(tt_data[3])

['cnbeta' '环球' '环球' 'cnbeta' '环球' 'cnbeta' 'cnbeta' 'cnbeta' '环球' 'cnbeta'
 '环球' 'cnbeta' 'cnbeta' 'cnbeta' 'cnbeta' '环球' 'cnbeta' 'cnbeta' 'cnbeta'
 'cnbeta' 'cnbeta' 'cnbeta' 'cnbeta' 'cnbeta' 'cnbeta' 'cnbeta' 'cnbeta'
 'cnbeta' 'cnbeta' 'cnbeta']
['cnbeta', '环球', '环球', 'cnbeta', '环球', 'cnbeta', 'cnbeta', '凤凰', '环球', 'cnbeta', '环球', '新浪', 'cnbeta', '凤凰', '参考消息', '环球', '参考消息', 'cnbeta', '参考消息', '参考消息', '环球', 'cnbeta', '参考消息', '凤凰', '参考消息', '环球', '参考消息', 'cnbeta', '参考消息', '参考消息']


In [75]:
def accuracy(t,p):
    tpc = 0
    for i in range(len(t)):
        if t[i] == p[i]:
            tpc = tpc + 1
    return float(tpc/len(t))

In [76]:
accuracy(tt_data[3], predict_rs)

0.5